# Recommendation systems

user_reviews.csv
- rows represent users of the system 
- columns represent products 
- cells represent a numerical rating from 1 to 5 of a product given by a user.
- value of 0 represents a missing rating—a movie the user hasn't rated yet.

movie_genres.csv
- rows represent the movies in the columns of  user_reviews.csv. 
- each row represents genre features of a single movie. 

create a reccomendation system to suggest which movies users should be recommended next that they haven't seen before. 

In [684]:
# # content based filtering
# -- watched by user, reccomend similar movies to user 

# may not have the breadth of movies that collaborative filtering may have
# more specific to each user 

# item to item collaborative filtering
# more users in a system than products (movies) in the System
# users are dynamic, things that they might like in their early teens they may not like when they are older

# item to itme method; item stays the same, regardless of tiem 

# measure angular distance cos theta 
# smaller angular distance, more similar the users taste are 
# if the movie ratings are exactly the same, should be one straight line

# # content based filtering
# -- watched by user, reccomend similar movies to user 

# may not have the breadth of movies that collaborative filtering may have
# more specific to each user 

# item to item collaborative filtering
# more users in a system than products (movies) in the System
# users are dynamic, things that they might like in their early teens they may not like when they are older

# item to itme method; item stays the same, regardless of tiem 

# measure angular distance cos theta 
# smaller angular distance, more similar the users taste are 
# if the movie ratings are exactly the same, should be one straight line

In [685]:
# user to user collaborative filtering

In [686]:
# import libraries
# use pandas to read data 
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [687]:
# Load data
data = pd.read_csv("user_reviews.csv", index_col=0)

# set User column as dataframe index
data = data.set_index('User')

In [688]:
data

,The Net,Happily N'Ever After,Tomorrowland,American Hero,Das Boot,Final Destination 3,Licence to Kill,The Hundred-Foot Journey,The Matrix,Creature,...,The Martian,Micmacs,Solomon and Sheba,In the Company of Men,Silent House,Big Fish,Get Real,Trading Places,DOA: Dead or Alive,Hey Arnold! The Movie
User,,,,,,,,,,,,,,,,,,,,,
Vincent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Edgar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Addilyn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Marlee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Javier,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mariana,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ivy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kevin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [689]:
# Predict the rating a user would give a movie
def baseline_prediction(data, user, movie_name):
    # overall average rating across all users and movies 
    global_mean = data.stack().mean()

    # mean rating user gives across all movies
    user_mean = data.loc[user, :].mean()
    
    # mean rating of movie across all users
    movie_mean = data.loc[:, movie_name].mean()
   
    # amount user's rating deviates from overall mean
    user_bias = global_mean - user_mean

    # amount rating for item deviates from overall mean
    item_bias = global_mean - movie_mean

    # predicted rating user gives to movie
    baseline_ui = global_mean + user_bias + item_bias
    
    return baseline_ui


In [690]:
# calculate the mean rating from all user for each movie
user_mean = data.mean(axis=0)
user_removed_mean_rating = (data - user_mean).fillna(0)

In [691]:
# find similar users using cosine similarity

def find_neighbor(user_removed_mean_rating, user, k=5):
    # Generate the similarity score
    n_users = len(user_removed_mean_rating.index)
    similarity_score = np.zeros(n_users)

    # get user 1 rating vector
    user_target = user_removed_mean_rating.loc[user].values.reshape(1, -1)

    # Iterate all users
    for i, neighbor in enumerate(user_removed_mean_rating.index):
        # Extract neighbor user vector
        user_neighbor = user_removed_mean_rating.loc[neighbor].values.reshape(1, -1)

        # Calculate the similarity (we use Cosine Similarity)
        sim_i = cosine_similarity(user_target, user_neighbor)

        # Append
        similarity_score[i] = sim_i

    # Sort in descending orders of similarity_score
    sorted_idx = np.argsort(similarity_score)[::-1]

    # sort similarity score, descending
    similarity_score = np.sort(similarity_score)[::-1]

    # get user closest neighbor
    closest_neighbor = user_removed_mean_rating.index[sorted_idx[1:k + 1]].tolist()

    # slice neighbour similarity
    neighbor_similarity = list(similarity_score[1:k + 1])

    # return closest_neighbor
    return {
        'closest_neighbor': closest_neighbor,
        'closest_neighbor_similarity': neighbor_similarity
    }

In [692]:
# Predict rating a user gives to a movie

def predict_item_rating(user, movie_name, data, neighbor_data, k, max_rating=5, min_rating=1):

    # calculate user's baseline rating for specific movie
    baseline_user = baseline_prediction(data=data, user=user, movie_name=movie_name)  # Correct variable name

    # for sum
    sim_rating_total = 0
    similarity_sum = 0

    # loop over all neighbors
    for i in range(k):
        
        # retrieve rating from neighbor
        neighbour_rating = data.loc[neighbor_data['closest_neighbor'][i], movie_name]

        # skip if rating is nan
        if np.isnan(neighbour_rating):
            continue

        # calculate neighbour's baseline rating for specific movie 
        baseline_neighbour = baseline_prediction(data=data, user=neighbor_data['closest_neighbor'][i], movie_name=movie_name)

        # subtract baseline from rating
        adjusted_rating = neighbour_rating - baseline_neighbour

        # multiply by similarity
        sim_rating = neighbor_data['closest_neighbor_similarity'][i] * adjusted_rating

        # sum similarity * rating
        sim_rating_total += sim_rating

        # sum similarity
        similarity_sum += neighbor_data['closest_neighbor_similarity'][i]

    # avoiding ZeroDivisionError
    try:
        user_item_predicted_rating = baseline_user + (sim_rating_total / similarity_sum)

    except ZeroDivisionError:
        user_item_predicted_rating = baseline_user

    # checking the boundaries of rating
    user_item_predicted_rating = max(min_rating, min(max_rating, user_item_predicted_rating))

    return user_item_predicted_rating


In [693]:
# Recommend items based on neighbours

def recommend_items(data, user, n_neighbor, n_items):
    # find neighbor
    neighbor_data = find_neighbor(user_removed_mean_rating=user_removed_mean_rating, user=user, k=n_neighbor)

    # create empty dataframe to store prediction result
    prediction_df = pd.DataFrame()

    # create list to store prediction result
    predicted_ratings = []

    # row of boolean values (True if unseen, False if seen)
    mask_unseen = (data.loc[user] == 0.0)  
    # row of unseen movies 
    item_to_predict = data.columns[mask_unseen]  

    # loop all over unseen movies
    for movie_name in item_to_predict:
        
        # predict rating
        preds = predict_item_rating(user=user, movie_name=movie_name, data=data, neighbor_data=neighbor_data, k=5)

        # append
        predicted_ratings.append(preds)

    # assign movie_name
    prediction_df['movie_name'] = item_to_predict

    # assign prediction result
    prediction_df['predicted_ratings'] = predicted_ratings

    # sort prediction result
    prediction_df = prediction_df.sort_values('predicted_ratings', ascending=False).head(n_items)

    return prediction_df


In [694]:
user_1_recommendation = recommend_items(data= data, user='Javier', n_neighbor=5, n_items=5)

print(user_1_recommendation)

                        movie_name  predicted_ratings
1915                    Life of Pi           1.548037
781               Last Action Hero           1.548037
1356        The Cabin in the Woods           1.548037
939   The Legend of Drunken Master           1.548037
743       Welcome to the Dollhouse           1.455565
